In [12]:
import csv
import pandas as pd
import xml.etree.ElementTree as ET
import xml.dom.minidom
from datasets import Dataset

full = pd.read_csv('aita_data.csv')

In [36]:
## Create dataset with just categorical vars
df = full.drop(columns = ['id', 'title', 'text', 'ratio', 'comments', 'url', 'op_age', 'target_age', 'length'])
df = df[['op_gender', 'target_gender', 'experience', 'relationship', 
         'industry', 'condition', 'action', 'intention', 
         'impact', 'good_standing', 'perspective', 'verdict']]

train_df  = df.sample(n=84, random_state=42)
test_indices = ~df.index.isin(train_df.index)
test_df = df[test_indices]

def classify_file(filename, df):
    with open(filename, 'w') as file:
        for _, row in df.iterrows():
            output = f"op_gender={row['op_gender']}," \
                     f"target_gender={row['target_gender']}," \
                     f"experience={row['experience']}," \
                     f"relationship={row['relationship']}," \
                     f"industry={row['industry']}," \
                     f"condition={row['condition']}," \
                     f"action={row['action']}," \
                     f"intention={row['intention']}," \
                     f"impact={row['impact']}," \
                     f"good_standing={row['good_standing']}," \
                     f"perspective={row['perspective']}," \
                     f"{row['verdict']}\n"

            file.write(output)

#classify_file('train.txt', train_df)
#classify_file('test.txt', test_df)

print(sum(train_df["verdict"]=="NTA")/len(train_df))

0.6071428571428571


In [4]:
## Same as above but with added features

def xml_file_features(df, xml_file):
    root = ET.Element("dataset")

    for index, row in df.iterrows():
        item = ET.SubElement(root, "item")
            
        item.set("label", str(row["verdict"]))
        item.set("title", str(row["title"]))
        item.set("op_gender", str(row["op_gender"]))
        item.set("target_gender", str(row["target_gender"]))
        item.set("experience", str(row["experience"]))
        item.set("relationship", str(row["relationship"]))
        item.set("industry", str(row["industry"]))
        item.set("condition", str(row["condition"]))
        item.set("action", str(row["action"]))
        item.set("intention", str(row["intention"]))
        item.set("impact", str(row["impact"]))
        item.set("good_standing", str(row["good_standing"]))
        item.set("perspective", str(row["perspective"]))

        content = ET.SubElement(item, "content")
        content.text = str(row["text"])

    tree = ET.ElementTree(root)

    xml_str = ET.tostring(root, encoding="unicode")

    dom = xml.dom.minidom.parseString(xml_str)
    pretty_xml_str = dom.toprettyxml()

    with open(xml_file, "w") as f:
        f.write(pretty_xml_str)

train_df  = full.sample(n=74, random_state=42)

tmp_indices = ~full.index.isin(train_df.index)
tmp = full[tmp_indices]

test_df = tmp.sample(n=20, random_state=42)

dev_indices = ~tmp.index.isin(test_df.index)
dev_df = tmp[dev_indices]


#xml_file_features(train_df, "train.xml")
#xml_file_features(test_df, "test.xml")
#xml_file_features(dev_df, "dev.xml")

In [23]:
dataset = pd.read_csv('aita_dataset.csv')

train_df  = dataset.sample(n=74, random_state=42)

tmp_indices = ~dataset.index.isin(train_df.index)
tmp = dataset[tmp_indices]

test_df = tmp.sample(n=20, random_state=42)

dev_indices = ~tmp.index.isin(test_df.index)
dev_df = tmp[dev_indices]

dataset_dict = {
    'train': train_df,
    'validation': dev_df,
    'test': test_df
}

In [31]:
# Load the DataFrame from CSV
df = pd.read_csv('aita_data_test.csv')

# Calculate overall accuracy
gpt_acc = sum(df["verdict"] == df["ChatGPT"]) / len(df) * 100
gem_acc = sum(df["verdict"] == df["Gemini"]) / len(df) * 100

# Calculate precision for NTA
gpt_pre_NTA = sum((df["verdict"] == "NTA") & (df["ChatGPT"] == "NTA")) / sum(df["ChatGPT"] == "NTA") * 100
gem_pre_NTA = sum((df["verdict"] == "NTA") & (df["Gemini"] == "NTA")) / sum(df["Gemini"] == "NTA") * 100

# Calculate precision for YTA
gpt_pre_YTA = sum((df["verdict"] == "YTA") & (df["ChatGPT"] == "YTA")) / sum(df["ChatGPT"] == "YTA") * 100
gem_pre_YTA = sum((df["verdict"] == "YTA") & (df["Gemini"] == "YTA")) / sum(df["Gemini"] == "YTA") * 100

# Calculate recall for NTA
gpt_rec_NTA = sum((df["verdict"] == "NTA") & (df["ChatGPT"] == "NTA")) / sum(df["verdict"] == "NTA") * 100
gem_rec_NTA = sum((df["verdict"] == "NTA") & (df["Gemini"] == "NTA")) / sum(df["verdict"] == "NTA") * 100

# Calculate recall for YTA
gpt_rec_YTA = sum((df["verdict"] == "YTA") & (df["ChatGPT"] == "YTA")) / sum(df["verdict"] == "YTA") * 100
gem_rec_YTA = sum((df["verdict"] == "YTA") & (df["Gemini"] == "YTA")) / sum(df["verdict"] == "YTA") * 100

# Calculate F1 score for NTA
gpt_f1_NTA = (2 * gpt_pre_NTA * gpt_rec_NTA) / (gpt_pre_NTA + gpt_rec_NTA)
gem_f1_NTA = (2 * gem_pre_NTA * gem_rec_NTA) / (gem_pre_NTA + gem_rec_NTA)

# Calculate F1 score for YTA
gpt_f1_YTA = (2 * gpt_pre_YTA * gpt_rec_YTA) / (gpt_pre_YTA + gpt_rec_YTA)
gem_f1_YTA = (2 * gem_pre_YTA * gem_rec_YTA) / (gem_pre_YTA + gem_rec_YTA)

# Print results
print("ChatGPT Overall Accuracy:", gpt_acc)
print("ChatGPT Precision NTA:", gpt_pre_NTA)
print("ChatGPT Precision YTA:", gpt_pre_YTA)
print("ChatGPT Recall NTA:", gpt_rec_NTA)
print("ChatGPT Recall YTA:", gpt_rec_YTA)
print("ChatGPT F1 NTA:", gpt_f1_NTA)
print("ChatGPT F1 YTA:", gpt_f1_YTA)

print("Gemini Overall Accuracy:", gem_acc)
print("Gemini Precision NTA:", gem_pre_NTA)
print("Gemini Precision YTA:", gem_pre_YTA)
print("Gemini Recall NTA:", gem_rec_NTA)
print("Gemini Recall YTA:", gem_rec_YTA)
print("Gemini F1 NTA:", gem_f1_NTA)
print("Gemini F1 YTA:", gem_f1_YTA)

ChatGPT Overall Accuracy: 61.904761904761905
ChatGPT Precision NTA: 88.88888888888889
ChatGPT Precision YTA: 41.66666666666667
ChatGPT Recall NTA: 53.333333333333336
ChatGPT Recall YTA: 83.33333333333334
ChatGPT F1 NTA: 66.66666666666667
ChatGPT F1 YTA: 55.555555555555564
Gemini Overall Accuracy: 61.904761904761905
Gemini Precision NTA: 81.81818181818183
Gemini Precision YTA: 40.0
Gemini Recall NTA: 60.0
Gemini Recall YTA: 66.66666666666666
Gemini F1 NTA: 69.23076923076924
Gemini F1 YTA: 49.99999999999999
